In [1]:
import os
import sys
from gensim.models import KeyedVectors

In [2]:
sys.path.append("sensegram_package/")
import sensegram
from wsd import WSD

In [3]:
data_directory = os.path.join(os.getcwd(), "data")
corpus_fpath = os.path.join(data_directory, "corpus.txt")
sense_vectors_fpath = os.path.join(data_directory, "model", "wiki.txt.clusters.minsize5-1000-sum-score-20.sense_vectors")
word_vectors_fpath = os.path.join(data_directory, "model", "wiki.txt.word_vectors")

In [4]:
if os.path.exists(sense_vectors_fpath) and os.path.exists(word_vectors_fpath):
    sense_vectors = sensegram.SenseGram.load_word2vec_format(sense_vectors_fpath, binary=False)
    word_vectors = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")
    wsd_model = WSD(sense_vectors, word_vectors, window=5, method='sim', filter_ctx=3)
else:
    print("Could not find vector files. Check file paths and ensure the right files exists")

NameError: name 'sense_vectors' is not defined

In [18]:
wsd_model = WSD(sence_vectors, word_vectors, window=15, method='sim', verbose=True)

In [7]:
with open(corpus_fpath, "r") as f:
    corpus_data = f.read()

#### Get all senses of a word

In [16]:
test_word = "table"

print("Probabilities of the senses:\n{}\n\n".format(sence_vectors.get_senses(test_word, ignore_case=True)))

for sense_id, prob in sence_vectors.get_senses(test_word, ignore_case=True):
    print(sense_id)
    print("="*20)
    for rsense_id, sim in sence_vectors.wv.most_similar(sense_id):
        print("{} {:f}".format(rsense_id, sim))
    print("\n")

Probabilities of the senses:
[('Table#1', 1.0), ('Table#2', 1.0), ('Table#3', 1.0), ('Table#4', 1.0), ('table#1', 1.0), ('table#2', 1.0), ('table#3', 1.0), ('table#4', 1.0)]


Table#1
table#1 0.996316
TABLE#1 0.993647
PAGE#2 0.989991
page#2 0.989991
WINDOW#2 0.989900
Window#3 0.989900
window#2 0.989900
Scale#2 0.989745
scale#2 0.989745
SCALE#2 0.989745


Table#2
TABLE#2 1.000000
Row#3 0.869726
row#3 0.869726
ROW#3 0.856643
Stack#3 0.829349
Box#3 0.826571
BOX#2 0.826571
stack#3 0.825068
STACK#3 0.824239
BOWL#3 0.813412


Table#3
TABLE#3 0.939938
table#3 0.934190
Boundary_Markers#5 0.845184
Catchment_Basins#2 0.826906
contents#2 0.825448
CONTENTS#2 0.825448
Contents#2 0.824324
tables#1 0.806271
NUMBERS#3 0.804637
Tables#1 0.796628


Table#4
TABLE#4 0.919513
KOI_Rp#4 0.912018
hlt_fst#2 0.909054
GB_NRT#2 0.905041
Algonquin_Round#1 0.905017
AYP_Data#2 0.904868
Sentry_Risk#1 0.900897
Peutinger#2 0.900774
Simplified_Characters#2 0.897952
Global_Descriptor#1 0.897648


table#1
Table#1 0.996316

#### Get disambiguated sense of the word, using corpus as context

In [20]:

wsd_model = WSD(sence_vectors, word_vectors, window=15, method='prob', verbose=True)

In [21]:
print(wsd_model.disambiguate(corpus_data, test_word))

Extracted context words:
['leftover', 'today', 'pizza', 'two', 'listening', 'english', 'saturday', 'just', 'ot', 'good', 'hangover', 'really', 'little', 'making', 'stop', 'slices']
Senses of a target word:
[('table#1', 1.0), ('table#2', 1.0), ('table#3', 1.0), ('table#4', 1.0)]
Significance scores of context words:
[0.37694023295858664, 0.5826593097827415, 0.7275173661671048, 0.3462219279577813, 0.32113965964726915, 0.6235405044059925, 0.314476217064492, 0.6094166324483048, 0.19088531161143885, 0.47508827794669944, 0.3897201415017436, 0.6215423778084821, 0.6857576958729588, 0.575916585269904, 0.7006624524726446, 0.7680371969757536]
Context words:
slices	0.768
pizza	0.728
stop	0.701
('table#2', [0.2706353009709064, 0.9591583572384959, 0.40617065436041355, 0.6940131864117054])


In [14]:
wsd_model.dis_text(corpus, test_word)

AttributeError: 'WSD' object has no attribute 'dis_text'